In [ ]:
!pip install simple-salesforce pandas

# 0 : Import the required libraries and define the Salesforce connection 

In [11]:
from simple_salesforce import Salesforce
import pandas as pd
config = pd.read_json('config.json')
target_org = Salesforce(username=config['target']['username'], password=config['target']['password'], security_token=config['target']['token'])
connected = target_org.session_id is not None
print('Connected to Target: '+str(connected))

Connected to Target: True


# 1: Read a CSV File using Pandas and convert the Dataframe into a dict (SimpleSalesforce manage the data as dicts)

In [12]:
account_df = pd.read_csv('accounts.csv')
#convert to Dict
records = account_df.to_dict('records')
print(records)

[{'Name': 'Orange', 'Phone': '123-456-7890'}, {'Name': 'Tricom', 'Phone': '987-654-3210'}, {'Name': 'Codetel', 'Phone': '555-123-4567'}]


# 2: Insert the data to the Target Org using the target_org connection

In [13]:
for account in records:
    target_org.Account.create(account)
    print('Account ' + account['Name'] + ' created successfully')

Account Orange created successfully
Account Tricom created successfully
Account Codetel created successfully


# 2.1: Wait there is something wrong with this... we are wasting many API Calls to insert the accounts... Let's fix it by using Bulk

In [14]:
result = target_org.bulk.Account.insert(records)
print(result)

[{'success': True, 'created': True, 'id': '0018b00002XIoNXAA1', 'errors': []}, {'success': True, 'created': True, 'id': '0018b00002XIoNYAA1', 'errors': []}, {'success': True, 'created': True, 'id': '0018b00002XIoNZAA1', 'errors': []}]


# 2.2: Even if Bulk is better... Bulkifying is recomended

In [15]:
chunk_size = 200
records_chunks = [] #list of lists

for i in range(0, len(records), chunk_size):
    chunk = records[i:i + chunk_size]
    records_chunks.append(chunk)

for chunk in records_chunks:
    target_org.bulk.Account.insert(chunk)
    print('Chunk inserted successfully')

Chunk inserted successfully
